In [ ]:
from nltk.corpus import gutenberg # to get bible corpus
from string import punctuation  # to remove punctuation from corpus
import nltk 
import numpy as np
from keras.preprocessing import text
from keras.preprocessing.sequence import skipgrams 
from keras.layers import *
from keras.layers.core import Dense, Reshape
from keras.layers.embeddings import Embedding
from keras.models import Model,Sequential 
from sklearn.decomposition import PCA
import re
import torch
import json
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.01)

In [ ]:
class GeneratorNet(torch.nn.Module):
    """
    A three hidden-layer generative neural network
    """
    def __init__(self):
        super(GeneratorNet, self).__init__()
        n_features = 768
        n_out = 768
        
        self.hidden1 = nn.Sequential(
            nn.Linear(n_features, 1024),
            nn.LeakyReLU(0.2)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.LeakyReLU(0.2)
        )
        
        self.out = nn.Sequential(
            nn.Linear(1024, n_out),
            nn.Tanh()
        )

        self.hidden1.apply(init_weights)
        self.hidden2.apply(init_weights)
        self.out.apply(init_weights)

    def forward(self, x):
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = 10*self.out(x)
        return x

In [ ]:
if torch.cuda.is_available():
    generator.cuda()
    # X = X.to('cuda')
    # Y = Y.to('cuda')
    print("good to go")

good to go


In [ ]:
from google.colab import drive
import os
import sys

drive.mount("/content/drive")


GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'EECS487NLP/project'
GOOGLE_DRIVE_PATH = os.path.join("drive", "My Drive", GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

# Add to sys so we can import .py files.
sys.path.append(GOOGLE_DRIVE_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['gendered_words-master', 'Gender-Debiasing-in-BERT-mybranch', 'Evaluation', 'context_nullspace_projection.py', 'natural_gendered_vocab.txt', 'nullspace_bert_demonstration.ipynb', 'generator_0.pt', 'eval_GAN.ipynb', 'bert-base-uncased-embeddings.txt', 'bert-base-uncased-embeddings (1).txt']


In [ ]:
generator = GeneratorNet()\
generator.load_state_dict(torch.load(f"{GOOGLE_DRIVE_PATH}/generator_0.pt"))

<All keys matched successfully>

In [ ]:
with open(f"{GOOGLE_DRIVE_PATH}/bert-base-uncased-embeddings.txt") as f:
  lines = f.readlines()
lines = lines[1:]


In [ ]:
from collections import defaultdict

In [ ]:
d = defaultdict(int)
for line in lines:
    split = line.split()
    key = split[0]
    value = split[1:]
    value = np.array(list(map(float, value)))
    d[key] = value

In [ ]:
generator = generator.double()

In [ ]:
for key, value in d.items(): 
    value = torch.from_numpy(value).double()
    # print(value.shape)
    if value.shape[0] == 768:
      d[key] = generator(value)
